In [32]:
import datetime as dt
from datetime import datetime, timedelta

In [ ]:
import duckdb

con = duckdb.connect()


con.execute("install httpfs")
con.execute("load httpfs")

In [4]:
con.sql(
    "create table gharchive as from read_json_auto('http://data.gharchive.org/2024-09-01-10.json.gz', ignore_errors=True)"
)

In [ ]:
con.sql("select * from gharchive limit 10").df()

In [ ]:
con.sql("select actor.* from gharchive limit 10").df()

In [10]:
import requests

gharchive_url = "http://data.gharchive.org/2024-09-01-10.json.gz"

response = requests.get(gharchive_url)

response_content = response.content

In [ ]:
response_content[:10]

In [30]:
import boto3
import dotenv
import io


env = dotenv.dotenv_values()

s3_client = boto3.client(
    "s3",
    aws_access_key_id=env["AWS_ACCESS_KEY"],
    aws_secret_access_key=env["AWS_SECRET_KEY"],
    region_name="us-east-2",
)


In [31]:
target_s3_key = "gharchive/events/2024-09-01-10.json.gz"
s3_client.upload_fileobj(
    io.BytesIO(response_content), "dataeng-landing-zone-957", target_s3_key
)

In [38]:
now = datetime.now(dt.timezone.utc)

# 2024-10-20 00:00:00+00:00
process_date = now.replace(minute=0, second=0, microsecond=0) - timedelta(hours=2)


In [39]:
process_date_str = process_date.strftime("%Y-%m-%d-%H")

In [40]:
url = f"https://data.gharchive.org/{process_date_str}.json.gz"
response = requests.get(url)
response.raise_for_status()

In [ ]:
response.content[:10]

In [ ]:
con.execute("install httpfs")

con.execute("load httpfs")

In [ ]:
import configparser


config = configparser.ConfigParser()

config.read("C:\\Users\\levyv\\Desktop\\projs\\duckdb_s3\\config.ini")

In [ ]:
con.execute(f"SET s3_access_key_id='{config['aws']['s3_access_key_id']}'")
con.execute(f"SET s3_secret_access_key='{config['aws']['s3_secret_access_key']}'")

In [ ]:
process_data = "2024-10-19"

con.sql(
    f"select * from 's3://dataeng-curated-zone-957/gharchive/events/2024-10-19/*.parquet'"
).to_table("gold")

In [80]:
gold_table = con.table("gold")

In [ ]:
gold_table.aggregate("count(*)").show()

In [ ]:
gold_table.aggregate("event_type, repo_name, sum(event_count) as event_count").filter(
    "event_type = 'WatchEvent'"
).order("event_count desc").limit(10).show()